In [1]:
import os
os.chdir(os.getcwd() + '/../../..')

In [2]:
from lib.python.crawler.selenium.chrome import BaseChromeWebdriver

import selenium
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import time

In [3]:
def get_topic_code(keyword: str, product_id: str) -> list[str]:
    """
    네트워크 패킷의 API 요청을 위한 토픽코드를 가져온다.
    
    Args:
        keyword (str): 수집을 원하는 키워드명.
        product_id (str): 수집을 원하는 상품번호.
    
    Return:
        list[str]: [토픽명,토픽코드]로 이루어진 리스트.
    """
    
    global topic_code_info, topic_url, saved_topic_url
    
    # 선택한 키워드에 해당하는 url로 접근한다.
    url = 'https://search.shopping.naver.com/catalog/{}?query={}'
    cwd = BaseChromeWebdriver(wired=True, chromedriver_path='tools/chromedriver', headless=True, timeout=None)
    cwd.get(url.format(product_id,keyword))

    # 스크롤 다운
    time.sleep(10)
    cwd.scroll_down()
    
    # 토픽 더보기 누르기
    selector = '#section_review > div.filter_sort_group__2G1QI > div.filter_evaluation_tap__V1J3G > div > a'
    elements = cwd.find_element_by_css_selector(selector)
    elements.send_keys(Keys.ENTER)
    
    # 토픽
    saved_topic_url = []
    topic_code_info = [['주제전체','']] # 토픽리스트 : 첫번째는 주제전체임

    topic_iter = 1
    while True:
        topic_iter += 1
        
        # 토픽명을 클릭해서 cwd.driver.requests에 url 정보가 쌓이도록 함
        try:
            # 토픽명 클릭
            selector_fmt = '#section_review > div.filter_sort_group__2G1QI > div.filter_evaluation_tap__V1J3G > div > ul > li:nth-child({}) > a'
            topic_elements = cwd.find_element_by_css_selector(selector_fmt.format(topic_iter))
            topic_elements.send_keys(Keys.ENTER)
            
            # 네트워크에 url나오도록 기다리기
            time.sleep(10)
            cwd.scroll_down()
            
        except NoSuchElementException as e:
            # 더 이상 토픽이 없음으로 인식
            break
        
        # url 가져오기
        topic_url = [request.url for request in cwd.driver.requests if request.url.find('https://search.shopping.naver.com/api/review?')>=0]
        topic_url = list(set(topic_url)-set(saved_topic_url))
        if len(topic_url)==1:
            saved_topic_url.append(topic_url[0])
        else:
            raise
        
        topic_code_info.append([topic_elements.text,topic_url[0].split('topicCode=')[-1]])
        
    return topic_code_info

In [4]:
keyword = '수분크림'
product_id = '8119197336'

In [5]:
topic_codes = get_topic_code(keyword,product_id)

In [6]:
topic_codes

[['주제전체', ''],
 ['만족도', 'total'],
 ['가격', 'price'],
 ['효과', 'effect'],
 ['트러블', 'trouble'],
 ['보습력', 'moisture'],
 ['성능', 'performance'],
 ['음식량', 'amount'],
 ['품질', 'quality'],
 ['흡수', 'absorption'],
 ['향기', 'smell'],
 ['편리성', 'usability'],
 ['사이즈', 'size'],
 ['색상', 'color'],
 ['디자인', 'design'],
 ['표현력', 'expression'],
 ['구성', 'component'],
 ['성분', 'ingredient'],
 ['지속력', 'keep'],
 ['제품상태', 'condition'],
 ['광택', 'gloss']]